# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
#drop unnecessary columns
new_df = df.drop('timestamp', axis=1)
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [4]:
from surprise import Reader, Dataset

# instantiate Reader
reader = Reader(rating_scale=(0, 5))

# load data
data = Dataset.load_from_df(new_df, reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency's sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [6]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [8]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors':[20,100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
grid.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  2.8min finished


In [9]:
# print out optimal parameters for SVD after GridSearch
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.890330699250083, 'mae': 0.6885499334153863}
{'rmse': {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, 'mae': {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}


In [11]:
# cross validating with KNNBasic
param_grid2 = {'k': [20, 40, 60],
              'sim_options': {'name': ['cosine', 'msd', 'pearson']}}

grid2 = GridSearchCV(KNNBasic, param_grid2, n_jobs=-1, joblib_verbose=5)
grid2.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/jennykreiger/anaconda3/envs/learn-env/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.5min finished


In [12]:
print(grid2.best_score)
print(grid2.best_params)

{'rmse': 0.9407522639346011, 'mae': 0.7185573982531338}
{'rmse': {'k': 20, 'sim_options': {'name': 'msd', 'user_based': True}}, 'mae': {'k': 20, 'sim_options': {'name': 'msd', 'user_based': True}}}


In [13]:
knn = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
knn_cv = cross_validate(knn, data, n_jobs=-1)

In [14]:
# print out the average RMSE score for the test set
for i in knn_cv.items():
    print(i)
print('-----')
print(np.mean(knn_cv['test_rmse']))

('test_rmse', array([0.97814714, 0.98039909, 0.96782576, 0.96903464, 0.96826106]))
('test_mae', array([0.75303123, 0.75766207, 0.74830464, 0.74640941, 0.74953901]))
('fit_time', (0.6010491847991943, 0.6458840370178223, 0.6911270618438721, 0.7270822525024414, 0.5571649074554443))
('test_time', (1.7629919052124023, 1.8581361770629883, 1.857116937637329, 1.693587303161621, 1.5305609703063965))
-----
0.972733536292326


In [15]:
# cross validating with KNNBaseline
knn_base = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
knn_base_cv = cross_validate(knn_base, data, n_jobs=-1)

In [16]:
# print out the average score for the test set
for i in knn_base_cv.items():
    print(i)
print('-----')
print(np.mean(knn_base_cv['test_rmse']))

('test_rmse', array([0.87546008, 0.86960876, 0.88497934, 0.87906675, 0.87827871]))
('test_mae', array([0.67118546, 0.66344061, 0.67314864, 0.6723243 , 0.67061853]))
('fit_time', (0.6806437969207764, 0.9608120918273926, 0.8920571804046631, 0.7949221134185791, 0.6375842094421387))
('test_time', (2.7450952529907227, 3.1253371238708496, 2.9490950107574463, 2.483642101287842, 2.2057809829711914))
-----
0.8774787301749013


Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [17]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [18]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [19]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [20]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=2.9589620782634443, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [26]:
def movie_rater(movie_df, num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list
        

In [27]:
# try out the new function here!
movie_rater(df_movies, 3)

     movieId                   title                  genres
643      829  Joe's Apartment (1996)  Comedy|Fantasy|Musical
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId             title                genres
5193     8454  Luna Papa (1999)  Comedy|Drama|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                                              title  genres
8758   128366  Patton Oswalt: Tragedy Plus Comedy Equals Time...  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                     title genres
2203     2928  Razor's Edge, The (1984)  Drama
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                            title  genres
2281     3028  Taming of the Shrew, The (1967)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId            

[{'userId': 1000, 'movieId': 2300, 'rating': '5'},
 {'userId': 1000, 'movieId': 8645, 'rating': ''},
 {'userId': 1000, 'movieId': 102802, 'rating': '3'}]

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [28]:
user_rating = [{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [29]:
## add the new ratings to the original ratings DataFrame
df_exp = new_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(df_exp, reader)

In [33]:
# train a model using the new combined DataFrame
knn_base = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
knn_base.fit(new_data.build_full_trainset())

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [34]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movie_list = []
for m_id in new_df['movieId'].unique():
    movie_list.append((m_id, knn_base.predict(1000, m_id)[3]))

In [35]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(movie_list, key=lambda x:x[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [36]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for index, rec in enumerate(user_ratings):
        title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
        print('Recommendation #', index+1, ':', title, '\n')    
        n -= 1
        if n == 0:
            break
recommended_movies(ranked_movies,df_movies,5)

Recommendation # 1 : 46    Usual Suspects, The (1995)
Name: title, dtype: object 

Recommendation # 2 : 89    Bottle Rocket (1996)
Name: title, dtype: object 

Recommendation # 3 : 257    Pulp Fiction (1994)
Name: title, dtype: object 

Recommendation # 4 : 314    Forrest Gump (1994)
Name: title, dtype: object 

Recommendation # 5 : 461    Schindler's List (1993)
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.